# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import csv

# Import API key
import sys
import os
key = "C:\\Users\\Kelly\\Desktop\\KEYS"
sys.path.append(key)
from OpenWeatherAPI import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)


url = "http://api.openweathermap.org/data/2.5/weather?"

## Generate Cities List

In [9]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name.title()
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

594

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [10]:
ident = []
name = []
weather = []

for city in cities:
    try:
        query_url = f"{url}q={city}&appid={weather_api_key}"
        weather_response = requests.get(query_url).json()
        ident.append(weather_response['id'])
        name.append(weather_response['name'])
        weather.append(weather_response['main'])
        index = cities.index(city)
        to_end = (len(cities))-index-1
        if to_end == 0:
            print(f'Now processing data for {weather_response["name"]}, city #{weather_response["id"]}.')
            print(f'All city data is now processed.')
        else:
            print(f'Now processing data for {weather_response["name"]}, city #{weather_response["id"]}. {to_end} more cities to process.')
    except:
        print(f'{city} was not found in our dataset, so we will not be collecting data for this city.')
print(weather)

Now processing data for Neiafu, city #4032420. 593 more cities to process.
Palabuhanratu was not found in our dataset, so we will not be collecting data for this city.
Now processing data for Pindwāra, city #1259638. 591 more cities to process.
Now processing data for Oranjemund, city #3354071. 590 more cities to process.
Now processing data for Hilo, city #5855927. 589 more cities to process.
Now processing data for Hobart, city #2163355. 588 more cities to process.
Now processing data for São Sebastião, city #3448257. 587 more cities to process.
Now processing data for Katsuura, city #2112309. 586 more cities to process.
Now processing data for Atuona, city #4020109. 585 more cities to process.
Now processing data for Ponta do Sol, city #2264557. 584 more cities to process.
Now processing data for Redwater, city #4722062. 583 more cities to process.
Now processing data for Palencia, city #3114530. 582 more cities to process.
Now processing data for Dewā, city #1273077. 581 more citie

Now processing data for Longyearbyen, city #2729907. 487 more cities to process.
Now processing data for Hermanus, city #3366880. 486 more cities to process.
Now processing data for Belyy Yar, city #1510377. 485 more cities to process.
Now processing data for Vostok, city #2013279. 484 more cities to process.
Now processing data for Llangefni, city #2644037. 483 more cities to process.
Now processing data for Upernavik, city #3418910. 482 more cities to process.
Now processing data for Mnogovershinnyy, city #2019935. 481 more cities to process.
Now processing data for Srednekolymsk, city #2121025. 480 more cities to process.
Now processing data for Lüderitz, city #3355672. 479 more cities to process.
Now processing data for Zaysan, city #1517060. 478 more cities to process.
Now processing data for Catalina, city #5288784. 477 more cities to process.
Now processing data for Botou, city #1816256. 476 more cities to process.
Now processing data for Cherskiy, city #2126199. 475 more cities

Now processing data for Port-de-Bouc, city #6447156. 382 more cities to process.
Now processing data for Omboué, city #2396853. 381 more cities to process.
Now processing data for Kirakira, city #2178753. 380 more cities to process.
Now processing data for Horsham, city #2646557. 379 more cities to process.
Now processing data for Mezen', city #527321. 378 more cities to process.
Now processing data for Marawi, city #1701054. 377 more cities to process.
Now processing data for New Norfolk, city #2155415. 376 more cities to process.
Now processing data for Kalmunai, city #1242110. 375 more cities to process.
Now processing data for Arraial do Cabo, city #3471451. 374 more cities to process.
Now processing data for Jalu, city #86049. 373 more cities to process.
Now processing data for Port Blair, city #1259385. 372 more cities to process.
Now processing data for Launceston, city #2160517. 371 more cities to process.
Now processing data for Husavik, city #2629833. 370 more cities to proce

Louisbourg was not found in our dataset, so we will not be collecting data for this city.
Saryshagan was not found in our dataset, so we will not be collecting data for this city.
Now processing data for Lorengau, city #2092164. 275 more cities to process.
Now processing data for Esqueda, city #4006532. 274 more cities to process.
Now processing data for Beihai, city #1816705. 273 more cities to process.
Now processing data for Sirte, city #2210554. 272 more cities to process.
Now processing data for Williamsburg, city #4793846. 271 more cities to process.
Now processing data for Cayenne, city #3382160. 270 more cities to process.
Now processing data for Monrovia, city #2274895. 269 more cities to process.
Now processing data for Waipawa, city #2206874. 268 more cities to process.
Meyungs was not found in our dataset, so we will not be collecting data for this city.
Nguiu was not found in our dataset, so we will not be collecting data for this city.
Now processing data for Catherinas L

Now processing data for Zapolyarnyy, city #464790. 172 more cities to process.
Now processing data for Teya, city #1489656. 171 more cities to process.
Now processing data for Korla, city #1529376. 170 more cities to process.
Now processing data for Plettenberg Bay, city #964712. 169 more cities to process.
Now processing data for Klichka, city #2021992. 168 more cities to process.
Now processing data for Pacific Grove, city #5380437. 167 more cities to process.
Now processing data for Cabo San Lucas, city #3985710. 166 more cities to process.
Now processing data for Port-Gentil, city #2396518. 165 more cities to process.
Now processing data for Samarinda, city #1629001. 164 more cities to process.
Now processing data for Kieta, city #2094027. 163 more cities to process.
Now processing data for Dakar, city #2253354. 162 more cities to process.
Now processing data for Killybegs, city #2963295. 161 more cities to process.
Now processing data for Tezu, city #1254709. 160 more cities to pr

Now processing data for Agua Verde, city #4019219. 66 more cities to process.
Now processing data for Mtwara, city #877747. 65 more cities to process.
Now processing data for Touros, city #3386213. 64 more cities to process.
Now processing data for Dhemāji, city #1272783. 63 more cities to process.
Now processing data for San Policarpo, city #1688696. 62 more cities to process.
Now processing data for Garoowe, city #58933. 61 more cities to process.
Now processing data for Cockburn Town, city #3576994. 60 more cities to process.
Urdzhar was not found in our dataset, so we will not be collecting data for this city.
Haibowan was not found in our dataset, so we will not be collecting data for this city.
Now processing data for Fukuechō, city #1863997. 57 more cities to process.
Now processing data for Pattoki, city #1168226. 56 more cities to process.
Now processing data for Vozrozhdeniye, city #866301. 55 more cities to process.
Now processing data for Hirara, city #1862505. 54 more citi

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [11]:
rows = zip(ident,name,weather)
with open("city_data.csv", "w", encoding="utf-8") as f:
    writer = csv.writer(f)
    for row in rows:
        writer.writerow(row)


In [24]:
ugly_df = pd.read_csv('city_data.csv', header = None)

ugly_df
#need to clean this up!

,0,1,2
0,4032420,Neiafu,"{'temp': 302.15, 'feels_like': 302.86, 'temp_m..."
1,1259638,Pindwāra,"{'temp': 297.69, 'feels_like': 294.22, 'temp_m..."
2,3354071,Oranjemund,"{'temp': 289.35, 'feels_like': 289.95, 'temp_m..."
3,5855927,Hilo,"{'temp': 294.94, 'feels_like': 295.74, 'temp_m..."
4,2163355,Hobart,"{'temp': 292.23, 'feels_like': 285.85, 'temp_m..."
...,...,...,...
542,536440,Lechinkay,"{'temp': 271.04, 'feels_like': 267.93, 'temp_m..."
543,286245,Sur,"{'temp': 297.2, 'feels_like': 292.57, 'temp_mi..."
544,2290462,Danané,"{'temp': 293.99, 'feels_like': 297.47, 'temp_m..."
545,4228452,Villa Rica,"{'temp': 280.24, 'feels_like': 277.47, 'temp_m..."


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [13]:
#  Get the indices of cities that have humidity over 100%.


In [14]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression